# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UDI peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__resultats_udi_communes
LIMIT 5

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00400121459,CHINE,0.000,"<=0,1 µg/L",2020,0.10000000149011612,µg/L,pesticides,004000129,04058,2020-04-09 11:49:00
00400121459,CHINOME,0.000,"<=0,1 µg/L",2020,0.10000000149011612,µg/L,pesticides,004000129,04058,2020-04-09 11:49:00
00400121459,CHLORB,0.000,"<=0,1 µg/L",2020,0.10000000149011612,µg/L,pesticides,004000129,04058,2020-04-09 11:49:00
00400121459,CHLPM,0.000,"<=0,1 µg/L",2020,0.10000000149011612,µg/L,pesticides,004000129,04058,2020-04-09 11:49:00
00400121459,CHLX,0.000,"<=0,1 µg/L",2020,0.10000000149011612,µg/L,pesticides,004000129,04058,2020-04-09 11:49:00


## Faire la liste complètes des communes 

In [3]:
%%sql --save list_communes
 
SELECT DISTINCT
    inseecommune 
 FROM
      int__lien_commune_cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune
53226
53273
54011
54020
54053
54070
54072
54073
54089
54091


## Test cog pour le final

In [4]:
%%sql 
SELECT 
    *
FROM 
  cog_communes
WHERE
    TYPECOM = 'COM'

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,de_partition,de_ingestion_date,de_dataset_datetime
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None,2024,2025-03-16,20240220
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None,2024,2025-03-16,20240220
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None,2024,2025-03-16,20240220
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None,2024,2025-03-16,20240220
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None,2024,2025-03-16,20240220
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None,2024,2025-03-16,20240220
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None,2024,2025-03-16,20240220
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None,2024,2025-03-16,20240220
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None,2024,2025-03-16,20240220
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None,2024,2025-03-16,20240220


## Les résultats par communes - Last prelevement 

###  SQL resultats_ALL_communes : Join avec list_communes_uid pour avoir la liste complète

###  Join avec cog pour avoir le nom des communes

# Final : ana__resultats_communes_last

In [2]:
%%sql 
WITH 
/* list_communes_uid  */
list_communes_uid AS (    
SELECT DISTINCT
    inseecommune 
 FROM
      int__lien_commune_cdreseau
),
/* END  list_communes_uid  */


/* communes_LAST_PVL */
 LAST_PVL AS (
    SELECT 
     *,
     ROW_NUMBER() OVER (PARTITION BY inseecommune, cdparametresiseeaux ORDER BY datetimeprel DESC ) As row_number
    FROM
      int__resultats_udi_communes
    WHERE 
        categorie= 'cvm' -- à supprimer pour avoir pour tout
    ),

communes_LAST_PVL AS (     
    SELECT 
     LAST_PVL.*  EXCLUDE (row_number)
    FROM
      LAST_PVL 
    WHERE
      row_number = 1
    ),
/* communes_LAST_PVL END */

    
resultats_all_communes_last AS ( 
    SELECT
        *
    FROM  
     list_communes_uid
    LEFT JOIN 
     communes_LAST_PVL
    ON
        list_communes_uid.inseecommune  =  communes_LAST_PVL.inseecommune
),

cog AS (
    SELECT 
      DEP AS code_departement,
      REG AS code_region,
      COM AS inseecommune,
      LIBELLE AS commune_nom
    FROM 
      cog_communes
    WHERE
        TYPECOM = 'COM'
)

SELECT
    'dernier prelevement' AS periode,
    cog.commune_nom,
    resultats_all_communes_last.*
FROM
  resultats_all_communes_last
LEFT JOIN  
  cog
ON 
  resultats_all_communes_last.inseecommune = cog.inseecommune


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

periode,commune_nom,inseecommune,referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune_1,datetimeprel
dernier prelevement,Xertigny,88530,08800185399,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,088001440,88530,2024-09-27 12:00:00
dernier prelevement,Montholon,89003,08900131165,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,089003823,89003,2024-06-26 10:22:00
dernier prelevement,Annay-la-Côte,89009,08900128872,CLVYL,0.004,<=0.5 µg/L,2023,0.5,µg/L,cvm,089000433,89009,2023-12-18 09:25:00
dernier prelevement,Annoux,89012,08900130376,CLVYL,0.005,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000714,89012,2024-04-16 10:48:00
dernier prelevement,Arcy-sur-Cure,89015,08900132364,CLVYL,0.010,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000434,89015,2024-10-04 09:13:00
dernier prelevement,Asnières-sous-Bois,89020,08900131810,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000693,89020,2024-08-21 09:23:00
dernier prelevement,Baon,89028,08900133222,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000443,89028,2024-12-17 09:58:00
dernier prelevement,Bernouil,89038,08900130521,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000725,89038,2024-05-06 11:28:00
dernier prelevement,Les Bordes,89051,08900133341,CLVYL,0.005,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000721,89051,2024-12-11 09:11:00
dernier prelevement,La Celle-Saint-Cyr,89063,08900133394,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,089000751,89063,2024-12-12 08:24:00
